In [57]:
import torch

x = torch.FloatTensor([[[1, 2],
                        [3, 4],
                        [5, 6]],

                        [[7, 8],
                        [9, 10],
                        [11, 12]],
                        
                        [[13, 14],
                        [15, 16],
                        [17, 18]]]) # (3,3,2)      

y = torch.FloatTensor([[[1, 2, 2],
                       [1, 2, 2]],
                       
                       [[1, 3, 3],
                       [1, 3, 3]],
                       
                       [[1, 4, 4],
                       [1, 4, 4]]]) # (3,2,3)





In [58]:
print(x.size(),y.size())

torch.Size([3, 3, 2]) torch.Size([3, 2, 3])


내적 규칙 : x의 열 수(2) == y 의 행 수(2)

In [59]:
z = torch.matmul(x, y) # 행렬 곱 수행 (마지막 2차원 기준)
print(z.size())

torch.Size([3, 3, 3])


- 마지막 2개 차원을 기준으로 행렬곱 수행
- batch 차원은 같아야 한다. (아니면 1이라도 되서 브로드캐스팅이라도 되어야 한다).

(3, 3, 2) * (3, 2, 3) = (3, 3, 3)

In [60]:
# 3차원에서만 사용할 수 있는 행렬곱 연산 BMM(batch matrix multiplication)
z = torch.matmul(x, y) # 행렬 곱 수행 (마지막 2차원 기준)
print(z.size()) 

torch.Size([3, 3, 3])


- torch.bmm
- 입력이 반드시 3차원 텐서 (B, N, M) @ (B, M, P) = (B, N, P)
- 각 batch는 batch마다 독립적으로 행렬곱 연산

| 구분 | torch.matmul | torch.bmm |
| --- |  -----|-----|
| 입력 차원 |  1D ~ ND | 3D 만 |
| batch 처리 | 자동 해석 + 브로드캐스팅 | batch 크기 동일 해야한다|
| 연산 범위 | 범용 행렬곱 | 배치 행렬곱 |
| 유연성 | 높다 | 낮다 |

bmm 은 3차원 배치 행렬곱 전용

matmul 은 차원 수에 따라 자동으로 처리되는 범용 행렬곱

.T (전치)를 이용해서 입력 차원과 곱셉이 가능하도록 맞춘다.

In [61]:
x = torch.tensor([[1., 2., 3.]])

W = torch.tensor([[0.1, 0.2, 0.3],
                  [0.4, 0.5, 0.6]])

print(x.size(),W.size()) # (1,3), (2,3)

torch.Size([1, 3]) torch.Size([2, 3])


In [62]:
W.T.size()

torch.Size([3, 2])

.T (전치 Transpose) : (2,3) -> (3,2)

In [63]:
y = torch.matmul(x,W.T) # (1, 3) @ (3, 2) -> (1, 2)
y.size()

torch.Size([1, 2])

In [64]:
y

tensor([[1.4000, 3.2000]])

행렬곱 절대원칙

행렬곱에서 마지막 두 차원은 반드시 아래와 같아야 한다.

(..., N, M) @ (..., M, P). => M == M

In [65]:
x = torch.FloatTensor([[[1, 2],
                        [3, 4],
                        [5, 6]],

                        [[7, 8],
                        [9, 10],
                        [11, 12]],
                        
                        [[13, 14],
                        [15, 16],
                        [17, 18]]])

W = torch.tensor([[0.1, 0.2, 0.3],
                  [0.4, 0.5, 0.6]])

print(x.size(),W.size())

torch.Size([3, 3, 2]) torch.Size([2, 3])


In [66]:
z =torch.matmul(x, W)
print(z.size())

torch.Size([3, 3, 3])


브로드캐스팅 (2, 3) -> batch 차원 추가 (1, 2, 3) -> batch 방향으로 복제 -> (3, 2, 3)

(3, 3, 2) @ (3, 2, 3) => (3, 3, 3)

In [67]:
x = torch.FloatTensor([[[1, 2],
                        [3, 4],
                        [5, 6]],

                        [[7, 8],
                        [9, 10],
                        [11, 12]],
                        
                        [[13, 14],
                        [15, 16],
                        [17, 18]]]) # (3,3,2)

W = torch.tensor([[[0.1, 0.2, 0.3],
                  [0.4, 0.5, 0.6]],

                  [[0.1, 0.2, 0.3],
                  [0.4, 0.5, 0.6]],

                  [[0.1, 0.2, 0.3],
                  [0.4, 0.5, 0.6]],

                  [[0.1, 0.2, 0.3],
                  [0.4, 0.5, 0.6]]]) # (4,2,3)

print(x.size(),W.size())

torch.Size([3, 3, 2]) torch.Size([4, 2, 3])


batch 차원이 3 !== 4 (맞지않음)   
내적 계산은 맞으나 자동으로 batch가 브로드캐스팅 되지않음

- 내적 차원 불일치 한 경우 : 불가
- batch 차원이 맞지 않는데 브로드캐스팅 조건에도 맞지 않는 경우 : 불가
- 내적 차원 일치 + batch 차원이 맞는 경우 : 가능 
- 내적 차원 일치 + batch 차원이 맞지 않는데, batch차원 브로드캐스팅 가능한 경우 : 가능 (자동 처리)  
=> batch는 행렬곱 연산 결과 자체에는 상관없으나, 형태가 맞아야 한다


In [68]:
x = torch.randn (2, 3, 2)
W = torch.randn (2, 4, 2)

# 3차원 이상은 transpose를 이용하여 명시적으로 교환 
y = torch.bmm(x, W.transpose(1, 2)) # (2, 3, 2) @ (2, 2, 4) => (2, 3, 4)
y.size()

torch.Size([2, 3, 4])

transpose(dim1, dim2) : 지정된 2개의 차원을 교환  
(2, 4, 2) -> (2, 2, 4) : 내적 차원 계산할수 있도록 전치

In [69]:
import torch
import torch.nn as nn

x = torch.tensor([[1., 2., 3.]])

W = torch.tensor([[0.1, 0.2, 0.3],
                  [0.4, 0.5, 0.6]])

print(x.size(),W.size()) # (1,3) (2,3)

torch.Size([1, 3]) torch.Size([2, 3])


nn.Linear 레이어 생성(bias 생략) : 수동 으로 만든 행렬 곱과 정확히 동일한 결과가 나온다.

In [70]:
linear = nn.Linear(
    in_features= 3,     # 입력 벡터의 길이 (x의 열), 행렬곱에서는 입력의 열 차원
    out_features= 2,    # 출력 벡터의 길이 (출력의 행), 
    bias = False        # bias 항 제외 -> 순수 행렬곱만 수행한 결과 
)

In [71]:
# W를 nn.Linear 레이어에 직접 복사
with torch.no_grad():       # 그래디언트 추적 비활성화 (가중치 수동 조절)
    linear.weight.copy_(W)  # Linear 레이어의 가중치를 W값으로 직접 덮어쓰기

copy_(W) : 대입해주는 게 아니라 값만 복사  
=> 파라미터 객체는 그대로 유지 

In [72]:
y2 = linear(x)          # 순전파 연산 (내부적으로는 행렬곱 계산만 진행)
print(y2,y2.size())     


tensor([[1.4000, 3.2000]], grad_fn=<MmBackward0>) torch.Size([1, 2])


linear (x) 연산을 했는데 내부적으로 자동으로 x @ W.T 시켜서 계산

In [74]:
import numpy as np